In [1]:
from openai import OpenAI
openai_client = OpenAI()

In [2]:
import requests
docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
docs_raw= docs_response.json()
documents =[]
for course in docs_raw:
    course_name = course['course']

    for docs in course['documents']:
        docs['course'] = course_name
        documents.append(docs)

In [3]:
print(documents[1])

{'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites', 'section': 'General course-related questions', 'question': 'Course - What are the prerequisites for this course?', 'course': 'data-engineering-zoomcamp'}


In [4]:
from minsearch import AppendableIndex
index = AppendableIndex(
    text_fields = ["question","text","section"],
    keyword_fields = ['course']
)
index.fit(documents)
    

In [5]:
def search(query):
    boost = {'question' : 3.0,'section': 0.5}

    results= index.search(
        query = query,
        filter_dict = {'course': 'data-engineering-zoomcamp'},
        boost_dict = boost,
        num_results= 5
    )
    return results

In [14]:
def make_call(call):
    args = json.loads(call.arguments)
    f_name = call.name

    if f_name == 'search':
        result = search(**args)
    else:
        raise Error(f'unknown function {f_name}')

    return {
        "type": "function_call_output",
        "call_id": call.call_id,
        "output": json.dumps(result),
    }

In [6]:
search_tool = {
    "type": "function",
    "name": "search",
    "description": "Search the FAQ database",
    "parameters": {
        "type": "object",
        "properties": {
            "query": {
                "type": "string",
                "description": "Search query text to look up in the course FAQ."
            }
        },
        "required": ["query"],
        "additionalProperties": False
    }
}        
                
        

In [7]:
tools = [search_tool]

In [8]:
instructions = """
You're a course teaching assistant. 
You're given a question from a course student and your task is to answer it.
If you want to look up the answer, explain why before making the call
""".strip()

In [9]:
question = 'I just discovered the course. Can I still join it?'

chat_messages = [
    {"role": "developer", "content": instructions},
    {"role": "user", "content": question}
]

In [10]:
#response.output

In [11]:
#response.output[0]

In [ ]:
while True: # the "agent" loop
    response = openai_client.responses.create(
        model='gpt-4o-mini',
        input=chat_messages,
        tools=tools
    )
    
    has_function_calls = False
    
    chat_messages.extend(response.output)
    
    for item in response.output:
        if item.type == 'message':
            print('Assistant:')
            print(item.content[0].text)
            print()
    
        if item.type == 'function_call':
            has_function_calls = True
            print('function call')
            print('    ', item.name, item.arguments)
            print()
            
            function_call_output = make_call(item)
            chat_messages.append(function_call_output)

    if not has_function_calls:
        break